---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [10]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

# Patterns

In [11]:
def date_sorter():
    
    import numpy as np, datetime as datetime

    p1 = r'((?P<Month>\d{1,2})[/|-](?P<Day>\d{1,2})[/|-](?P<Year>[^3]\d{1,3}))'
    p2 = r'((?P<Day>\d{2})[-,.]? (?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[.]?[-,.]? (?P<Year>\d{4}))'
    p3 = r'((?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[.]?[-,.]?[- ]?(?P<Day>\d{2})[-,.]?[ ]?(?P<Year>\d{4}))'
    p4 = r'((?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*?)[,-]? (?P<Year>[12]\d{3}))'
    p5 = r'((?P<Month>\d{1,2})/(?P<Year>\d{4}))'
    p6 = r'((?P<Year>[12]\d{3}))'

    df_A = df.str.extractall(p1)
    df_A_ix = [x[0] for x in df_A.index.values]
    df_A['Year'] = df_A['Year'].str.replace(r'(^\d{2}$)', lambda x: '19'+ x.groups()[0])
    df_A['Month'] = df_A['Month'].str.replace(r'(^0\d{1}$)', lambda x: x.groups()[0][1:])
    df_A = df_A[['Year','Month','Day']]

    df_B = df.iloc[df_A_ix[-1]+1:].str.extractall(p2)
    df_B_ix = [x[0] for x in df_B.index.values]
    df_B['Month'] = df_B['Month'].str.replace(r'(\w+)',lambda x: x.groups()[0][:3])
    df_B['Month'] = df_B['Month'].replace({'Jan':1, 'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,
                        'Jul':7,'Aug':8,'Sep':9,'Oct':10, 'Nov':11, 'Dec':12})
    df_B = df_B[['Year','Month','Day']]

    df_C = df.iloc[df_B_ix[-1]+1:].str.extractall(p3)
    df_C_ix = [x[0] for x in df_C.index.values]
    df_C['Month'] = df_C['Month'].str.replace(r'(\w+)',lambda x: x.groups()[0][:3])
    df_C['Month'] = df_C['Month'].replace({'Jan':1, 'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,
                        'Jul':7,'Aug':8,'Sep':9,'Oct':10, 'Nov':11, 'Dec':12})
    df_C = df_C[['Year','Month','Day']]

    df_D = df.iloc[df_C_ix[-1]+1:].str.extractall(p4)
    df_D_ix = [x[0] for x in df_D.index.values]
    df_D['Month'] = df_D['Month'].str.replace(r'(\w+)',lambda x: x.groups()[0][:3])
    df_D['Month'] = df_D['Month'].replace({'Jan':1, 'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,
                        'Jul':7,'Aug':8,'Sep':9,'Oct':10, 'Nov':11, 'Dec':12})
    df_D['Day'] = 1
    df_D = df_D[['Year','Month','Day']]

    df_E = df.iloc[df_D_ix[-1]+1:].str.extractall(p5)
    df_E_ix = [x[0] for x in df_E.index.values]
    df_E['Day'] = 1
    df_E = df_E[['Year','Month','Day']]

    df_F = df.iloc[df_E_ix[-1]+1:].str.extractall(p6)
    df_F_ix = [x[0] for x in df_F.index.values]
    df_F['Month'] = 1
    df_F['Day'] = 1
    df_F = df_F[['Year','Month','Day']]

    df_full = pd.concat([df_A, df_B, df_C, df_D, df_E, df_F])
    df_time = pd.to_datetime(df_full)

    dfsort = pd.DataFrame({'rank': df_time.index.levels[0], 'time':df_time.values})
    dfsort.sort_values('time', inplace=True)

    
    
    return dfsort['rank']

In [14]:
date_sorter()

9        9
84      84
2        2
53      53
28      28
474    474
153    153
13      13
129    129
98      98
111    111
225    225
31      31
171    171
191    191
486    486
335    335
415    415
36      36
405    405
323    323
422    422
375    375
380    380
345    345
57      57
481    481
436    436
104    104
299    299
      ... 
220    220
208    208
243    243
139    139
320    320
383    383
244    244
286    286
480    480
431    431
279    279
198    198
381    381
463    463
366    366
439    439
255    255
401    401
475    475
257    257
152    152
235    235
464    464
253    253
427    427
231    231
141    141
186    186
161    161
413    413
Name: rank, Length: 500, dtype: int64